<a href="https://colab.research.google.com/github/iqbal-apriza/pythonProject/blob/from_colab/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def convolution(data1, data2):
    data1_dim = data1.shape
    data2_dim = data2.shape
    result = np.zeros((data1_dim[0] - data2_dim[0] + 1, data1_dim[1] - data2_dim[1] + 1))
    for i in range(data1_dim[0] - data2_dim[0] + 1):
        for j in range(data1_dim[1] - data2_dim[1] + 1):
            result[i, j] = np.sum(data1[i:i+data2_dim[0], j:j+data2_dim[1]] * data2)
    return result

def relu(data):
    data_dim = data.shape
    for i in range(data_dim[0]):
        for j in range(data_dim[1]):
            if data[i, j] < 0:
                data[i, j] = 0
    return data

def max_pooling(data, size):
    data_dim = data.shape
    row = int(data_dim[0] / size)
    col = int(data_dim[1] / size)
    result = np.zeros((row, col))
    for i in range(row):
        for j in range(col):
            result[i, j] = data[i*2:i*2+size, j*2:j*2+size].max()
    return result

def flat(data):
    return data.flatten()

In [3]:
dataa = np.array([
    [0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0]
])

kernel = np.array([
    [1, 1],
    [1, 0]
])

targets = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],      # 0
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],      # 1
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],      # 2
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],      # 3
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],      # 4
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],      # 5
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],      # 6
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],      # 7
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],      # 8
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]       # 9
]

dataa_dim = dataa.shape
kernel_dim = kernel.shape

In [4]:
test = convolution(dataa, kernel)
test = relu(test)
test = max_pooling(test, 2)
test = flat(test)
print(test)

[2. 2. 2. 2.]


In [ ]:
result = np.zeros((dataa_dim[0] - kernel_dim[0] + 1, dataa_dim[1] - kernel_dim[1] + 1))
for i in range(dataa_dim[0] - kernel_dim[0] + 1):
    for j in range(dataa_dim[1] - kernel_dim[1] + 1):
        conv = np.sum(dataa[i:i+kernel_dim[0], j:j+kernel_dim[1]] * kernel)
        result[i, j] = conv

print(result)

[[0. 2. 1. 1.]
 [1. 2. 1. 2.]
 [1. 2. 1. 2.]
 [1. 1. 2. 1.]]


In [ ]:
print(dataa[0:2, 2:4])
print(test)

[[1 0]
 [0 1]]
None


In [ ]:
inputt = np.array([
    [1, 1],
    [1, 0],
    [0, 1],
    [0, 0]
])

output = np.array([0, 1, 1, 0])

weight_v = np.array([
    [ 0.2,  0.3, -0.1],
    [ 0.3,  0.1, -0.1],
    [-0.3,  0.3,  0.3]
])

weight_w = np.array([
    [ 0.5],
    [-0.3],
    [-0.4],
    [-0.1]
])

learn_rate = 0.999

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def layer_calc(data, weight):
    w_dim = weight.shape
    out = np.zeros((w_dim[1]))
    for i in range(w_dim[1]):
        out[i] = np.sum(data * weight[0:w_dim[0]-1, i]) + weight[w_dim[0]-1, i]
        out[i] = sigmoid(out[i])
    return out

def err_calc(target, output):
    return target - output

def d_out(d_net, output):
    out_dim = output.shape
    res = np.zeros(out_dim[0])
    for i in range(out_dim[0]):
        res[i] = d_net[i] * output[i] * (1 - output[i])
    return res

def d_error(d_out, weight):
    w_dim = weight.shape
    res = np.zeros(w_dim[0]-1)
    for i in range(w_dim[0]-1):
        res[i] = d_out * weight[i]
    return res

def w_change(alp, d_out, inp):
    in_dim = inp.shape
    d_dim = d_out.shape
    res = np.zeros((in_dim[0]+1, d_dim[0]))
    for i in range(d_dim[0]):
        for j in range(in_dim[0]):
            res[j, i] = alp * d_out[i] * inp[j]
        res[in_dim[0], i] = alp * d_out[i]
    return res

def w_update(weight, d_w):
    return weight + d_w

In [ ]:
data_dim = inputt.shape
error = np.zeros((data_dim[0], 1))
for epoch in range(500):
    for i in range(data_dim[0]):
        # Forward propagation
        layer1 = layer_calc(inputt[i], weight_v)
        layer2 = layer_calc(layer1, weight_w)

        # Error calc
        error[i] = err_calc(output[i], layer2)

        # Backward propagation
        d_lay2 = d_out(error[i], layer2)
        d_out1 = d_error(d_lay2, weight_w)
        d_lay1 = d_out(d_out1, layer1)

        # Weight change
        w_ch = w_change(learn_rate, d_lay2, layer1)
        v_ch = w_change(learn_rate, d_lay1, inputt[i])
        weight_w = w_update(weight_w, w_ch)
        weight_v = w_update(weight_v, v_ch)

    mse = np.mean(error ** 2)
    print(mse)

In [ ]:
for i in range(data_dim[0]):
    # Forward propagation
    layer1 = layer_calc(inputt[i], weight_v)
    layer2 = layer_calc(layer1, weight_w)
    print(layer2)

[0.05063533]
[0.95378746]
[0.97253131]
[0.03137281]
